In [39]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('Alessi_Teutsch_5')
print(latex_text(cl))

\subsection{Introduction}
    Alessi Teutsch 5 is located at $\alpha = 332^\circ16{}^\prime27.84{}^{\prime\prime}, \delta = 61^\circ05{}^\prime46.32{}^{\prime\prime}$ at a distance of $870.0\pm31.0$ pc.
    59 out of the 147 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:Alessi_Teutsch_5_kinematics}. Some of the brightest members of the cluster are shown in table \ref{tab:Alessi_Teutsch_5_members}.
    
    \subsection{CMD and proper motion}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{fig:Alessi_Teutsch_5_cmd_pm} in the left.
    The runaway star is highlighted with a temperature estimate of 11060 using this isochrone. This corresponds to a spectral class of ... .
    The right plot in the figure shows the proper motion diagram showing that the ru

In [6]:
plot_cmd = cl.plot_cmd()
show_annotation_manager(plot_cmd)

In [12]:
plot_pm = cl.plot_pm()
show_annotation_manager(plot_pm)

In [38]:
plot_traceback_clean = cl.plot_traceback_clean()
show_annotation_manager(plot_traceback_clean)

10pc image downloaded in 8.334120s


In [ ]:
plot_traceback_psr = cl.plot_traceback_psr()


In [29]:
cl.psrs(sep_limit=600*u.arcmin)

JNAME,RAJD,DECJD,DIST,DIST_DM,AGE,PMRA,PMDEC,S400,ASSOC,AGE_I,PX,P0,P1,BSURF,SkyCoord,sep2d,sep3d
,deg,deg,kpc,kpc,yr,mas / yr,mas / yr,mJy,,yr,mas,s,,G,"deg,deg,kpc",deg,kpc
str12,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,SkyCoord,float64,float64
J2326+6141,351.49999999999994,61.0,1.695,1.695,--,--,--,--,--,--,--,0.79,--,--,"351.49999999999994,61.0,1.695",9.273641332940834,0.8489871436667646
